# Solving elasticity problems using JuliaFEM

Author(s): Jukka Aho

**Abstract**: Solving elasticity equations using JuliaFEM.

### Weak form

Given function spaces
\begin{align}
\boldsymbol{\mathcal{U}} & =\left\{ \boldsymbol{u}\in H^{1}\left(\Omega\right)|\boldsymbol{u}\left(\boldsymbol{X},t\right)=\hat{\boldsymbol{u}}\left(\boldsymbol{X},t\right)\text{ on }\Gamma_{\mathrm{u}}\right\} ,\\
\boldsymbol{\mathcal{V}} & =\left\{ \delta\boldsymbol{u}\in H^{1}\left(\Omega\right)|\delta\boldsymbol{u}\left(\boldsymbol{X}\right)=0\text{ on }\Gamma_{\mathrm{u}}\right\} ,
\end{align}
find $\boldsymbol{u}\in\boldsymbol{\mathcal{U}}$ such that
\begin{equation}
\delta\mathcal{W}:=\int_{\Omega_{0}}\rho_{0}\ddot{\boldsymbol{u}}\cdot\delta\boldsymbol{u}\,\mathrm{d}V_{0}+\int_{\Omega_{0}}\boldsymbol{S}:\delta\boldsymbol{E}\,\mathrm{d}V_{0}-\int_{\Omega_{0}}\hat{\boldsymbol{b}}_{0}\cdot\delta\boldsymbol{u}\,\mathrm{d}V_{0}-\int_{\Gamma_{\sigma}}\hat{\boldsymbol{t}}_{0}\cdot\delta\boldsymbol{u}\,\mathrm{d}A_{0} =0 \qquad\forall\delta\boldsymbol{u}\in\boldsymbol{\mathcal{V}}
\end{equation}

### Some formulas
\begin{align}
J & =\det\left(F\right)\\
I_{c} & =\mbox{tr}\left(C\right)\\
\mathbf{C} & =\mathbf{F}^{\mathrm{T}}\mathbf{F}\\
\mathbf{F} & =\mathbf{I}+\nabla\mathbf{u}\\
\mathbf{E} & =\frac{1}{2}\left(\mathbf{F}^{\mathrm{T}}\mathbf{F}-\mathbf{I}\right)
\end{align}

### Potential energy

\begin{equation}
\underset{u\in\boldsymbol{\mathcal{U}}}{\min}\Pi\left(\mathbf{u}\right)
\end{equation}
\begin{equation}
\Pi\left(\mathbf{u}\right)=\int_{\Omega}\psi\left(\mathbf{u}\right)-\int_{\Omega}\hat{\mathbf{b}}_{0}\cdot\mathbf{u}-\int_{\Gamma_{\sigma}}\hat{\mathbf{t}}_{0}\cdot\mathbf{u}\,\mathrm{d}A_{0}
\end{equation}

### Material models

https://en.wikipedia.org/wiki/Strain_energy_density_function

Saint-Venant-Kirchhoff model https://en.wikipedia.org/wiki/Hyperelastic_material
\begin{equation}
\psi\left(\mathbf{E}\right)=\frac{\lambda}{2}\left[\mbox{tr}\left(\mathbf{E}\right)\right]^{2}+\mu\mbox{tr}\left(\mathbf{E}^2\right)
\end{equation}

neo-Hookean material https://en.wikipedia.org/wiki/Neo-Hookean_solid
\begin{equation}
\psi=\frac{\mu}{2}\left(I_{c}-3\right)-\mu\ln\left(J\right)+\frac{\lambda}{2}\ln\left(J\right)^{2}
\end{equation}

In [1]:
using JuliaFEM

In [2]:
model = open(JuliaFEM.parse_abaqus, "../geometry/piston/piston_8789_P1.inp")
model

INFO: Registered handlers: Any["ELEMENT","NODE","NSET","ELSET"]
INFO: Parsing elements
INFO: 31437 elements found
INFO: Creating ELSET PISTON


Dict{Any,Any} with 3 entries:
  "nodes"    => Dict{Any,Any}(2843=>[-27.41338,-2.18093,-16.93988],4495=>[-14.0…
  "elements" => Dict{Any,Any}(43367=>[4492,4602,4494,6450],35510=>[2796,2798,80…
  "elsets"   => Dict{Any,Any}("BC3"=>[3975,3976,3977,3978,3979,3980,3981,3982,3…

INFO: Parsing elements
INFO: 5894 elements found
INFO: Creating element set BC1
INFO: Creating element set BC2
INFO: Creating element set BC3


In [3]:
perm = Dict{Int64, Int64}()
for (i, nid) in enumerate(keys(model["nodes"]))
    perm[nid] = i
end

In [4]:
using JuliaFEM: Element, Tri3, Tet4
elements = Dict{Int64, Element}()
for (elid, node_ids) in model["elements"]
    connectivity = Int64[perm[nid] for nid in node_ids]
    coords = Vector{Float64}[model["nodes"][nid] for nid in node_ids]
    if length(coords) == 3
        element = Tri3(connectivity)
        element["geometry"] = coords
        elements[elid] = element
    elseif length(coords) == 4
        element = Tet4(connectivity)
        element["geometry"] = coords
        elements[elid] = element
    else
        warn("unknown element dim = $(length(connectivity))")
    end
end
info("$(length(elements)) elements created.")

INFO: 37331 elements created.


In [5]:
31437+5894

37331

In [6]:
using JuliaFEM: ElasticityProblem, DirichletProblem

piston = ElasticityProblem()
for elid in model["elsets"]["PISTON"]
    elements[elid]["youngs modulus"] = 210.0e3
    elements[elid]["poissons ratio"] = 0.3
    push!(piston, elements[elid])
end

In [7]:
load = Vector{Float64}[[0.0, 0.0, -10.0], [0.0, 0.0, -10.0], [0.0, 0.0, -10.0]]
for elid in model["elsets"]["BC1"]
    elements[elid]["displacement traction force"] = load
    push!(piston, elements[elid])
end

In [8]:
encastre = DirichletProblem("displacement", 3)
for elid in model["elsets"]["BC2"]
    elements[elid]["displacement"] = 0.0
    push!(encastre, elements[elid])
end

In [9]:
using JuliaFEM: DirectSolver
solver = DirectSolver()
push!(solver, piston)
push!(solver, encastre);

In [10]:
solver(0.0)

INFO: # of field problems: 1
INFO: # of boundary problems: 1
INFO: Starting iteration 1
INFO: # of dofs: 26367, # of interface dofs: 1350
INFO: solved. length of solution vector = 52734
INFO: timing info for non-linear iteration:
INFO: boundary assembly      : 1.4367611408233643
INFO: field assembly         : 17.380300998687744
INFO: create sparse matrices : 0.38140082359313965
INFO: solution of system     : 3.159921169281006
INFO: update element data    : 3.3958511352539062
INFO: non-linear iteration   : 25.75425100326538
INFO: Starting iteration 2
INFO: # of dofs: 26367, # of interface dofs: 1350
INFO: solved. length of solution vector = 52734
INFO: timing info for non-linear iteration:
INFO: boundary assembly      : 0.1028587818145752
INFO: field assembly         : 15.321197032928467
INFO: create sparse matrices : 0.11679887771606445
INFO: solution of system     : 2.1216089725494385
INFO: update element data    : 3.484022855758667
INFO: non-linear iteration   : 21.146512031555176
IN

(4,true)

## Saving results to file

In [11]:
xdoc, xmodel = JuliaFEM.xdmf_new_model()
temporal_collection = JuliaFEM.xdmf_new_temporal_collection(xmodel)
grid = JuliaFEM.xdmf_new_grid(temporal_collection; time=0)

<Grid Name="Grid">
  <Time Value="0"/>
</Grid>


Save geometry to xdmf file

In [12]:
nnodes = length(model["nodes"])
info("number of nodes in model: $nnodes")

X = zeros(3, nnodes)
for nid in keys(model["nodes"])
    X[:, perm[nid]] = model["nodes"][nid]
end

nelements = length(model["elsets"]["PISTON"])
info("Number of elements in model: $nelements")
elmap = zeros(Int64, 5, nelements)
#elmap[1,:] = 0x0026
elmap[1,:] = 0x6
for (i, elid) in enumerate(model["elsets"]["PISTON"])
    elmap[2:end,i] = Int64[perm[nid] for nid in model["elements"][elid]]
end

INFO: number of nodes in model: 8789
INFO: Number of elements in model: 31437


In [13]:
JuliaFEM.xdmf_new_mesh(grid, X, elmap)

true

In [14]:
JuliaFEM.xdmf_save_model(xdoc, "/tmp/piston.xmf")

988949

Save nodal data to model

In [15]:
using JuliaFEM: get_connectivity

u = zeros(3, nnodes)

for element in values(elements)
    isa(element, Element{Tet4}) || continue
    connectivity = get_connectivity(element)
    field = element["displacement"](0.0)
    for (i, nid) in enumerate(connectivity)
        u[:, nid] = field[i]
    end
end
size(u)

(3,8789)

In [16]:
JuliaFEM.xdmf_new_field(grid, "Displacement", "nodes", u)

true

In [17]:
JuliaFEM.xdmf_save_model(xdoc, "/tmp/piston.xmf")

1573751